In [ ]:
# install necessary packages (works on Google Colab even if you see a versioning error)
root = "/tmp/XCP-explain"
! git clone https://github.com/CPMpy/XCP-explain.git {root}
! cd {root} && git checkout ecai24
! pip install -r /tmp/XCP-explain/requirements.txt -qq  # -qq=very quiet

import sys
if root not in sys.path:
    sys.path.insert(0, root)
print("Ready to go!")

In [ ]:
# load necessary packages
import networkx as nx
import cpmpy as cp

# graph drawing functions
draw = lambda g,**kwargs : nx.draw_circular(g, width=4, node_size=500,**kwargs)
cmap = ["black", "yellow", "cyan", "lightgreen", "blue", "red", "magenta", "orange", "purple", "brown"]

def graph_highlight(graph, cons, dotted=False, **kwargs):
    edges = []
    for c in cons:
        n1, n2 = c.args
        if n1.name == "max": continue
        a = int(re.search("\[[0-9]*\]", str(n1)).group()[1:-1])
        b = int(re.search("\[[0-9]*\]", str(n2)).group()[1:-1])
        edges.append((a,b))
        
    colors = ["red" if (a,b) in edges else "black" for (a,b) in graph.edges()]
    linestyles = ["dotted" if c == "red" and dotted else "solid" for c in colors]
    return draw(graph, edge_color=colors, style=linestyles, **kwargs)

## Let's generate a graph

Play with the parameters to get a different one!

In [ ]:
G = nx.fast_gnp_random_graph(7, 0.5, seed=21)
draw(G)

## How many colors do you think it needs?

In [ ]:
max_colors = 3  # YOUR GUESS FOR YOUR GRAPH

def graph_color_k(G, max_colors):
    # number of nodes
    n = G.number_of_nodes()

    # decision variables, one for every node
    x = cp.intvar(1, max_colors, shape=n, name="x")

    # constraints: neighbors have different colors
    m = cp.Model(
        [x[i] != x[j] for i,j in G.edges()],
    )
    return m, x

m, nodes = graph_color_k(G, max_colors)
if m.solve():
    print(m.status())
    print(f"Yes! There is a coloring with {max(nodes.value())} <= {max_colors} colors:")
    draw(G, node_color=[cmap[v.value()] for v in nodes])
else:
    print("No solution found, let the expaining begin : )")

## Given too few colors, lets find an explanation

In [ ]:
assert not m.solve(), "Choose too few colors to continue"